1. Sanity checks of dataset before merging

In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [92]:
purchase = pd.read_csv('/home/celinaha/ml_group21/data/kernel/purchase_orders.csv')

In [93]:
(purchase.select_dtypes(include=['number'])<0).any()

purchase_order_id         False
purchase_order_item_no    False
quantity                   True
product_id                False
product_version           False
unit_id                   False
status_id                 False
dtype: bool

General info:

In [94]:
purchase.shape

(33171, 12)

In [95]:
purchase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33171 entries, 0 to 33170
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   purchase_order_id       33171 non-null  int64  
 1   purchase_order_item_no  33171 non-null  int64  
 2   quantity                33171 non-null  float64
 3   delivery_date           33171 non-null  object 
 4   product_id              33171 non-null  int64  
 5   product_version         33171 non-null  int64  
 6   created_date_time       33171 non-null  object 
 7   modified_date_time      32679 non-null  object 
 8   unit_id                 33127 non-null  float64
 9   unit                    33127 non-null  object 
 10  status_id               33171 non-null  int64  
 11  status                  33171 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 3.0+ MB


In [96]:
# unique rows
purchase.nunique()

purchase_order_id          8135
purchase_order_item_no       94
quantity                   4997
delivery_date              2332
product_id                   77
product_version               9
created_date_time         14657
modified_date_time        22845
unit_id                       2
unit                          2
status_id                     3
status                        3
dtype: int64

In [97]:
# missing values:

purchase.isnull().sum()

purchase_order_id           0
purchase_order_item_no      0
quantity                    0
delivery_date               0
product_id                  0
product_version             0
created_date_time           0
modified_date_time        492
unit_id                    44
unit                       44
status_id                   0
status                      0
dtype: int64

In [98]:
purchase.isnull().sum()/purchase.shape[0]*100

purchase_order_id         0.000000
purchase_order_item_no    0.000000
quantity                  0.000000
delivery_date             0.000000
product_id                0.000000
product_version           0.000000
created_date_time         0.000000
modified_date_time        1.483223
unit_id                   0.132646
unit                      0.132646
status_id                 0.000000
status                    0.000000
dtype: float64

In [99]:
# duplicates
int(purchase.duplicated().sum())

0

In [100]:
# identifying garbage values
for i in purchase.select_dtypes(include="object").columns:
    print(purchase[i].value_counts())
    print("***"*10)

delivery_date
2014-12-31 00:00:00.0000000 +01:00    284
2009-09-30 00:00:00.0000000 +02:00    264
2009-11-30 00:00:00.0000000 +01:00    258
2009-07-31 00:00:00.0000000 +02:00    257
2009-03-31 00:00:00.0000000 +02:00    253
                                     ... 
2024-09-16 00:00:00.0000000 +02:00      1
2024-09-24 00:00:00.0000000 +02:00      1
2003-02-10 00:00:00.0000000 +01:00      1
2004-08-20 00:00:00.0000000 +02:00      1
2004-08-27 00:00:00.0000000 +02:00      1
Name: count, Length: 2332, dtype: int64
******************************
created_date_time
2014-03-12 15:03:36.0000000 +00:00    32
2015-03-19 15:13:19.0000000 +00:00    32
2014-11-14 12:50:47.0000000 +00:00    31
2012-10-30 12:24:49.0000000 +00:00    30
2006-12-14 17:33:01.0000000 +00:00    30
                                      ..
2004-10-22 12:21:54.0000000 +00:00     1
2005-03-11 13:53:25.0000000 +00:00     1
2006-03-27 11:04:44.0000000 +00:00     1
2012-07-04 13:53:29.0000000 +00:00     1
2024-12-27 09:53:17.00000

CHANGING DATATYPES

3️⃣ Recommendation for your dataset

    * Before merging / EDA: convert IDs to str — easier for plotting, counting, aggregation.

    * Before modeling: convert IDs to category for tree-based models, or use encoding (one-hot, target, frequency) if needed.

    * Avoid astype(str) on columns with missing values.
**********************************************************************************************************************************************
    * Use astype('string') for EDA or astype('category') for modeling.

    * Missing IDs are important — don’t silently convert them to "nan".

In [101]:
#datetime
for i in purchase.columns:
    if "date" in i:
        purchase[i] = pd.to_datetime(purchase[i], utc=True, errors='coerce')

In [102]:
#strings
for i in purchase.columns:
    if i.endswith("_id") or i.endswith("_no"):
        purchase[i] = purchase[i].astype('string')

In [103]:
# unit, and project_version
purchase['unit'] = purchase['unit'].astype("string")
purchase['product_version'] = purchase['product_version'].astype("string")

In [104]:
purchase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33171 entries, 0 to 33170
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   purchase_order_id       33171 non-null  string             
 1   purchase_order_item_no  33171 non-null  string             
 2   quantity                33171 non-null  float64            
 3   delivery_date           33171 non-null  datetime64[ns, UTC]
 4   product_id              33171 non-null  string             
 5   product_version         33171 non-null  string             
 6   created_date_time       33171 non-null  datetime64[ns, UTC]
 7   modified_date_time      32679 non-null  datetime64[ns, UTC]
 8   unit_id                 33127 non-null  string             
 9   unit                    33127 non-null  string             
 10  status_id               33171 non-null  string             
 11  status                  33171 non-null  o

Checking impossible values

In [105]:
# negative quantities
purchase[purchase['quantity']<0]


,purchase_order_id,purchase_order_item_no,quantity,delivery_date,product_id,product_version,created_date_time,modified_date_time,unit_id,unit,status_id,status
0,1,1,-14.0,2003-05-11 22:00:00+00:00,91900143,1,2003-05-12 10:00:48+00:00,2004-06-15 06:16:18+00:00,<NA>,<NA>,2,Closed
59,205323,10,-380.0,2003-01-30 23:00:00+00:00,1024,1,2003-01-28 13:30:23+00:00,2004-06-15 06:08:40+00:00,40.0,KG,2,Closed
60,205443,10,-390.0,2003-02-09 23:00:00+00:00,1024,1,2003-02-10 09:31:21+00:00,2004-06-15 06:14:46+00:00,40.0,KG,2,Closed
61,206151,10,-260.0,2003-04-20 22:00:00+00:00,1024,1,2003-04-14 09:31:14+00:00,2004-06-15 06:14:50+00:00,40.0,KG,2,Closed
66,206386,10,-340.0,2003-05-13 22:00:00+00:00,1024,1,2003-05-09 08:30:53+00:00,2004-06-15 06:14:54+00:00,40.0,KG,2,Closed
208,209842,10,-8580.0,2004-04-29 22:00:00+00:00,91900170,1,2004-04-28 08:32:31+00:00,2004-06-15 06:16:36+00:00,40.0,KG,2,Closed


In [109]:
purchase.isna().sum()

purchase_order_id           0
purchase_order_item_no      0
quantity                    0
delivery_date               0
product_id                  0
product_version             0
created_date_time           0
modified_date_time        492
unit_id                    44
unit                       44
status_id                   0
status                      0
dtype: int64

First EDA steps; .describe()

In [112]:
purchase.describe().T

,count,mean,std,min,25%,50%,75%,max
quantity,33171.0,90314.155619,330809.698715,-8580.0,10000.0,26280.0,100000.0,24975988.0


In [113]:
purchase.describe(include='string').T

,count,unique,top,freq
purchase_order_id,33171,8135,231471,93
purchase_order_item_no,33171,94,10,8118
product_id,33171,77,91900143,7115
product_version,33171,9,1,25685
unit_id,33127,2,40.0,33120
unit,33127,2,KG,33120
status_id,33171,3,2,28780
